Шаблон для запуска парсинга: \
python parsing.py 2 True housedorf urls_housedorf.txt example.xlsx result_housedorf.xlsx 
python parsing.py 2 False falmec urls_falmec.txt result_housedorf.xlsx result.xlsx

python generate_syn_report.py housedorf syn_report_housedorf.xlsx
python generate_syn_report.py falmec syn_report_falmec.xlsx

python compare.py falmec housedorf

# Программа parsing.py

In [2]:
import urllib3
from bs4 import BeautifulSoup, NavigableString
import pandas as pd
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill, Alignment
import re
from tqdm import tqdm
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import argparse
import os
from ruwordnet import RuWordNet
import pymorphy2
from itertools import product
from pathlib import Path

synonyms_path='synonyms.txt'
all_characteristics_path='all_characteristics.xlsx'

In [10]:
# ------------------------------------------------------------------------Спарсить данные------------------------------------------------------------------------

# Функции для парсинга
def parse_korting_page(html_code):
    soup = BeautifulSoup(html_code, 'html.parser')
    tabs_lists = soup.find_all('ul', class_='tabs-settings__list')
    data = {}

    for ul in tabs_lists:
        for li in ul.find_all('li'):
            text = li.get_text(strip=True, separator="; ")
            split_text = text.split(":;", 1)
            if len(split_text) == 2:
                key, value = split_text
                data[key.strip()] = value.strip()
            else:
                data[split_text[0].strip()] = ""

    return data

def parse_dedietrich_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}

    # Проходим по всем div с классом characteristics__row
    for row in soup.find_all('div', class_='characteristics__row'):
        name_span = row.find('span', class_='characteristics__name')
        value_span = row.find('span', class_='characteristics__property')
        
        if name_span and value_span:
            key = name_span.find(text=True, recursive=False)
            value = value_span.find(text=True, recursive=False)
            if key and value:
                key = key.strip()
                value = value.strip()
                data[key] = value

    return data

def parse_vzug_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}
    names = soup.find_all('td', class_='cell_name')
    values = soup.find_all('td', class_='cell_value')

    # Проходим по всем div с классом characteristics__row
    for name, value in zip(names, values):
        name_span = name.find('span')
        value_span = value.find('span')
        
        if name_span and value_span:
            key = name_span.find(text=True, recursive=False)
            value = value_span.find(text=True, recursive=False)
            if key and value:
                key = key.strip()
                value = value.strip()
                data[key] = value

    return data

def parse_falmec_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}

    # Проходим по всем div с классом characteristics__row
    for row in soup.find_all('div', class_='characteristics__row'):
        name_span = row.find('span', class_='characteristics__name')
        value_span = row.find('span', class_='characteristics__property')
        
        if name_span and value_span:
            key = name_span.find(text=True, recursive=False)
            value = value_span.find(text=True, recursive=False)

            # Если нет простого текста, ищем <ul> и собираем <li>
            if (value == '' or not value.strip()) and value_span.find('ul'):
                li_items = value_span.find_all('li')
                value = '; '.join(li.get_text(strip=True) for li in li_items)

            if key and value:
                key = key.strip()
                value = value.strip()
                data[key] = value

    return data

def extract_first_visible_text(tag):
    for desc in tag.descendants:
        if isinstance(desc, str):  # Это NavigableString
            text = desc.strip()
            if text:
                return text
    return None

def clean_value_div(value_div):
    # 1. Удалить все <span>
    for span in value_div.find_all("span"):
        span.decompose()

    # 2. Разделить по <br> — создаём список на основе HTML с разделителем
    parts = str(value_div).split('<br')

    values = []

    for part_html in parts:
        # Восстанавливаем HTML-тег <br>, если он был отрезан
        if not part_html.startswith('>'):
            part_html = '<br' + part_html

        part_soup = BeautifulSoup(part_html, 'html.parser')

        # 3. Найти первый видимый текст
        for desc in part_soup.descendants:
            if isinstance(desc, NavigableString):
                text = desc.strip()
                if text:
                    values.append(text)
                    break  # только первое вхождение

    # 4. Склеить с разделителем "; "
    return "; ".join(values)

def parse_hausedorf_page(html_code):
    soup = BeautifulSoup(html_code, 'html.parser')
    fields = soup.find_all('div', class_='detail-properties__field')
    data = {}

    for field in fields:
        name_div = field.find('div', class_='detail-properties__name')
        value_div = field.find('div', class_='detail-properties__value')

        if name_div and value_div:
            key = extract_first_visible_text(name_div)
            value = clean_value_div(value_div)

            if key and value:
                data[re.sub(r'\s+', ' ', key).strip()] = value.replace(">\n", "")

    return data


def create_src(file_path, parser_func):
    """
    Универсальный загрузчик таблицы характеристик с разных сайтов.

    :param file_path: путь к файлу с URL (один URL на строку)
    :param parser_func: функция, которая получает HTML-код и возвращает словарь {ключ: значение}
    :return: DataFrame с объединёнными результатами
    """
    http = urllib3.PoolManager()
    df_all = pd.DataFrame()

    with open(file_path, 'r', encoding='utf-8') as f:
        urls = [line.strip() for line in f]

    n_rows = 0
    for url in tqdm(urls):
        if url:
            try:
                response = http.request('GET', url)
                html_code = response.data.decode()
                data = parser_func(html_code) 

                if not isinstance(data, dict):
                    raise ValueError("parser_func должна возвращать словарь!")

                row_df = pd.DataFrame([data])
                df_all = pd.concat([df_all, row_df], ignore_index=True)

                if len(df_all) == 1:
                    empty_rows = pd.DataFrame(np.nan, index=range(n_rows), columns=df_all.columns)
                    df_all = pd.concat([empty_rows, df_all], ignore_index=True)

            except Exception as e:
                print(f"Ошибка при обработке {url}: {e}")
        else:
            if len(df_all.columns) > 0:                
                df_all.loc[len(df_all)] = None
            else:
                n_rows += 1

    return df_all.where(pd.notnull(df_all), None)

# ---------------------------------------------Записать и вернуть дополненный названиями номенклатуры DataFrame---------------------------------------------

def write_dest(ref_file_path, result_file_path, df_src, start_row_index):
    # Путь к файлу Excel
    wb = load_workbook(ref_file_path)
    ws = wb.active  # или wb['SheetName']

    # Поля файла-приёмника
    row_header = [cell.value for cell in ws[1]]

    # Сопоставление колонок
    src_cols_lower = {col.lower(): col for col in df_src.columns}
    ws_cols_lower = {i: str(header).strip().lower() if header else "" for i, header in enumerate(row_header)}
    matched_columns = []
    common_cols = set()
    nomenclature_col_idx = None

    for col_idx, header_lower in ws_cols_lower.items():
        if header_lower == "номенклатура":
            nomenclature_col_idx = col_idx
        if header_lower in src_cols_lower:
            matched_columns.append((col_idx, src_cols_lower[header_lower]))
            common_cols.add(src_cols_lower[header_lower])

    if nomenclature_col_idx is None:
        raise ValueError("Колонка 'Номенклатура' не найдена в файле-приёмнике")

    # Считываем значения "Номенклатура"
    nomenclature_values = []
    for i in range(len(df_src)):
        cell_value = ws.cell(row=start_row_index + i, column=nomenclature_col_idx + 1).value
        nomenclature_values.append(cell_value)

    # Запись данных
    for i, (_, row_src) in enumerate(df_src.iterrows()):
        for col_idx, src_col in matched_columns:
            cell = ws.cell(row=start_row_index + i, column=col_idx + 1)
            if cell.value in [None, ""]:
                cell.value = row_src[src_col]

    # Сохранение
    wb.save(result_file_path)

    # Подготовка выходного DataFrame
    result_df = df_src[list(common_cols)].copy()
    result_df.insert(0, "Номенклатура", pd.Series(nomenclature_values))

    return result_df, common_cols

# -------------------------------------------Сохранить незаписанные данные в дополнительные колонки или отдельный файл-------------------------------------------

def append_dataframe_to_excel(df: pd.DataFrame, file_path: str, result_path: str, start_row: int):
    # Проверка, существует ли файл
    if os.path.exists(file_path):
        wb = load_workbook(file_path)
    else:
        # Если файла нет, создаем новый
        wb = Workbook()
    
    ws = wb.active

    # Найдём первую пустую ячейку в первой строке
    col_index = 1
    while ws.cell(row=1, column=col_index).value is not None:
        col_index += 1

    # Записываем названия колонок DataFrame в первую строку, начиная с найденной колонки
    for i, col_name in enumerate(df.columns):
        ws.cell(row=1, column=col_index + i, value=col_name)

    # Автонастройка ширины столбцов по первой строке
    for col_idx, cell in enumerate(ws[1], start=col_index):
        max_length = len(str(cell.value)) if cell.value else 0
        col_letter = cell.column_letter
        ws.column_dimensions[col_letter].width = max_length + 2  # +2 для отступа

    # Применение стилей и переносов
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row, max_col=ws.max_column):
        for cell in row:
            cell.alignment = Alignment(wrap_text=True, vertical='center')  # включаем перенос текста

    # Записываем данные DataFrame начиная со start_row
    for row_offset, row in enumerate(dataframe_to_rows(df, index=False, header=False)):
        for i, value in enumerate(row):
            ws.cell(row=start_row + row_offset, column=col_index + i, value=value)

    # Сохраняем файл
    wb.save(result_path)

def save_missing(df1, filepath):

    # Создаём ExcelWriter
    with pd.ExcelWriter(filepath, engine='openpyxl') as writer:
        row = 0

        # Запись df1
        df1.to_excel(writer, index=False, startrow=row)
        row += len(df1) + 2  # +1 за заголовок, +1 за пустую строку

        # Автоматическая установка ширины колонок
        worksheet = writer.sheets['Sheet1']
        for column_cells in worksheet.columns:
            max_length = 0
            column = column_cells[0].column
            for cell in column_cells:
                if cell.value:
                    max_length = max(max_length, len(str(cell.value)))
            adjusted_width = max_length + 2
            worksheet.column_dimensions[get_column_letter(column)].width = adjusted_width

In [4]:
# Протестировать функцию парсинга
def test_parser(url, parser_func):
    http = urllib3.PoolManager()
    df_all = pd.DataFrame()
    response = http.request('GET', url)
    html_code = response.data.decode()
    data = parser_func(html_code)
    return data

data = test_parser('https://vzug-shop.ru/catalog/kholodilniki/vstraivaemaya-morozilnaya-kamera-v-zug-freezer-v4000-178n-fr4t-53003/', parse_vzug_page)
print(len(data))  # Должно вернуть количество характеристик, например 10
data # Выводим результат парсинга

28


{'Серия': 'V4000',
 'Модель': 'FR4T-53003',
 'Тип прибора': 'Встраиваемая морозильная камера',
 'Страна бренда': 'Швейцария',
 'Изготовитель': 'Фауцуг',
 'Гарантия (кол-во лет)': '2',
 'Ширина, мм': '600',
 'Высота, мм': '1778',
 'Цвет': 'Белый',
 'Дизайнерский фасад': 'Под навес вашего фасада',
 'Ручка': 'Без ручки',
 'Тип управления': 'TouchControl',
 'Дисплей': 'Цифровой дисплей',
 'Объем, л': '213',
 'Полезный объем морозильной камеры, л': '213',
 'Размораживание морозильной камеры': 'No Frost',
 'Механизм закрывания дверцы SoftClose/SoftClosePlus': 'Есть',
 'Количество корзин, контейнеров, ящиков': '8',
 'Дверные петли': 'Слева',
 'Класс энергоэффективности': 'E',
 'Уровень шума, Дб': '36',
 'Климатический класс': 'SN-T',
 'Длина сетевого кабеля, см': '220',
 'Тип штекера': 'Schuko 16A',
 'Габариты, мм': '1770х559х546',
 'Габариты ниши для встраивания ВхШхГ, мм': '1780-1788х560-570х550',
 'Глубина с открытой дверью, мм': '1137',
 'Вес, кг': '82.5'}

In [5]:
# ---------------------------------------------------------Добавить функции для сопоставления синонимов---------------------------------------------------------

wordnet = RuWordNet()
morph = pymorphy2.MorphAnalyzer()

def get_normal_form(word):
    return morph.parse(word)[0].normal_form

def are_synonyms(word1, word2):
    lemma1 = get_normal_form(word1)
    lemma2 = get_normal_form(word2)

    synsets1 = wordnet.get_synsets(lemma1)
    synsets2 = wordnet.get_synsets(lemma2)

    # Сравниваем наличие общих лемм в синсетах
    for s1 in synsets1:
        for s2 in synsets2:
            if s1.id == s2.id:
                return True
    return False

def list_synonyms_comparison(list1, list2):
    return [are_synonyms(word1, word2) for word1, word2 in zip(list1, list2)]

In [15]:
# Протестировать подбор синонимов
list1 = ['Машина', 'Счастливый', 'Работать']
list2 = ['автомобиля', 'счастливый', 'работник']
list_synonyms_comparison(list1, list2)

[True, True, False]

In [6]:
# -----------------------------Добавить функцию для обеспечения правильного дописывания данных основываясь на утверждённых синонимах-----------------------------

def parse_custom_dict_line(line):
    """
    Разбирает строку из словаря: <характеристика>: <синоним1>; <синоним2>; ...| <антисиноним1>, <антисиноним2>, ...
    """
    base, *rest = line.strip().split(':')
    if not rest:
        return base.strip(), set(), set()
    syn_ant = rest[0].split('|')
    synonyms = set(map(str.strip, syn_ant[0].split(';'))) if syn_ant[0] else set()
    antonyms = set(map(str.strip, syn_ant[1].split(','))) if len(syn_ant) > 1 else set()
    return base.strip(), synonyms, antonyms

def load_existing_synonyms(file_path):
    syn_dict = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            base, synonyms, antonyms = parse_custom_dict_line(line)
            syn_dict[base] = {'synonyms': synonyms, 'antonyms': antonyms}
    return syn_dict

def rename_columns_with_syn_dict(df, syn_dict_path, all_1c_chars_path):
    # Загрузка словаря
    synonyms_dict = load_existing_synonyms(syn_dict_path)
    all_chars = pd.read_excel(all_1c_chars_path, header=None).iloc[0].astype(str).tolist()
    all_chars_lower = [char.lower() for char in all_chars]  # Приводим к нижнему регистру для сравнения

    # Удаление дублирующихся колонок по имени
    df = df.loc[:, ~df.columns.duplicated()]

    # Поиск синонимичных имен колонок через словарь
    df_expanded = df.copy()
    unsyn_set = set()
    for col in df.columns:
        flag = False
        for synonym_key, syn_data in synonyms_dict.items():
            syn_set = syn_data.get("synonyms", set())
            if col in syn_set:
                # Добавляем колонку с именем synonym_key, если она отличается и ещё не существует
                if synonym_key != col and synonym_key not in df_expanded.columns:
                    df_expanded[synonym_key] = df[col]
                    flag = True
        if flag:
            # Если колонка была переименована, удаляем оригинальную
            df_expanded.drop(columns=[col], inplace=True)
        elif col.lower() in all_chars_lower:
            df_expanded[col.upper()] = df[col]
            df_expanded.drop(columns=[col], inplace=True)
        else:
            unsyn_set.add(col)

    return df_expanded, unsyn_set

# Варианты запуска парсинга

In [ ]:
# -----------------------------------------------------------Запуск парсинга и сохранение результатов-----------------------------------------------------------

# Протестировать запись одной строки с двух сайтов
from argparse import Namespace

# housedorf
args = Namespace(start_row=33, append=True, site='housedorf', urls_source='input_example/url_housedorf.txt', input_path='input_example/example.xlsx', output_path='result_housedorf.xlsx')
if not args.output_path:
    raise ValueError("there's not enough arguments")

# Проверка, не повреждены ли входные файлы
wb = load_workbook(args.input_path)

if args.site == 'korting':
    df_src = create_src(args.urls_source, parse_korting_page)
elif args.site == 'housedorf':
    df_src = create_src(args.urls_source, parse_hausedorf_page)
elif args.site == 'dedietrich':
    df_src = create_src(args.urls_source, parse_dedietrich_page)
elif args.site == 'falmec':
    df_src = create_src(args.urls_source, parse_falmec_page)
elif args.site == 'vzug':
    df_src = create_src(args.urls_source, parse_vzug_page)
else:
    raise ValueError("There're no parse function for this site")

df_src, unsyn_set = rename_columns_with_syn_dict(df_src, synonyms_path, all_characteristics_path) # Для обеспечения правильного дописывания данных и для корректного входа к функции, 
                                                                        # генерирующей отчёт
resultdf, com_cols = write_dest(args.input_path, args.output_path, df_src, args.start_row)
resultdf.to_parquet(f"{args.site}_auxiliary.parquet")

com_cols = resultdf.columns.intersection(df_src.columns)
missingdf = df_src.drop(columns=com_cols).copy()
print(unsyn_set)
with open(f'unaccepted_syn_{args.site}.txt', 'w') as f:
    f.write('; '.join(map(str, unsyn_set)))

if args.append:
    append_dataframe_to_excel(missingdf, args.output_path, args.output_path, args.start_row)
else:
    missingdf.insert(0, 'Номенклатура', resultdf['Номенклатура'].copy())
    save_missing(missingdf, f'missing_{args.site}.xlsx')

# korting
args = Namespace(start_row=34, append=False, site='korting', urls_source='input_example/url_korting.txt', input_path='result_housedorf.xlsx', output_path='result.xlsx')
if not args.output_path:
    raise ValueError("there's not enough arguments")

# Проверка, не повреждены ли входные файлы
wb = load_workbook(args.input_path)

if args.site == 'korting':
    df_src = create_src(args.urls_source, parse_korting_page)
elif args.site == 'housedorf':
    df_src = create_src(args.urls_source, parse_hausedorf_page)
elif args.site == 'dedietrich':
    df_src = create_src(args.urls_source, parse_dedietrich_page)
elif args.site == 'falmec':
    df_src = create_src(args.urls_source, parse_falmec_page)
elif args.site == 'vzug':
    df_src = create_src(args.urls_source, parse_vzug_page)
else:
    raise ValueError("There're no parse function for this site")

df_src, unsyn_set = rename_columns_with_syn_dict(df_src, synonyms_path, all_characteristics_path) # Для обеспечения правильного дописывания данных и для корректного входа к функции, 
                                                                        # генерирующей отчёт
resultdf, com_cols = write_dest(args.input_path, args.output_path, df_src, args.start_row)
resultdf.to_parquet(f"{args.site}_auxiliary.parquet")

com_cols = resultdf.columns.intersection(df_src.columns)
missingdf = df_src.drop(columns=com_cols).copy()
print(unsyn_set)
with open(f'unaccepted_syn_{args.site}.txt', 'w') as f:
    f.write('; '.join(map(str, unsyn_set)))

if args.append:
    append_dataframe_to_excel(missingdf, args.output_path, args.output_path, args.start_row)
else:
    missingdf.insert(0, 'Номенклатура', resultdf['Номенклатура'].copy())
    save_missing(missingdf, f'missing_{args.site}.xlsx')

100%|██████████| 2/2 [00:03<00:00,  1.70s/it]


,Тип,Вид,Высота (см),Ширина (см),Глубина (см),Габариты (ВхШхГ) (см),Габариты ниши для встраивания (ВхШхГ) (см),Количество камер,Количество дверей,Дверной упор,...,Мощность подключения (Вт),Уровень шума (дб),Длина шнура электропитания (м),Вес нетто (кг),Возможность перевешивания двери,Монтаж двери,Регулируемые ножки,Индикаторы,Индикация открытой двери,Артикул
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Встраиваемый,Морозильник,178.5,54,54.5,178.5х54х54.5,177.6-178.4х56-56.5х56,1,1,Слева,...,65,41,1.975,67,Да,Скользящее крепление двери (система door sliding),Да,Температуры морозильного отделения,Да,00000020289


In [ ]:
# dedietrich + housedorf
from argparse import Namespace

# housedorf
args = Namespace(start_row=2, append=True, site='housedorf', urls_source='urls_housedorf.txt', input_path='example.xlsx', output_path='result_housedorf.xlsx')
if not args.output_path:
    raise ValueError("there's not enough arguments")

# Проверка, не повреждены ли входные файлы
wb = load_workbook(args.input_path)

if args.site == 'korting':
    df_src = create_src(args.urls_source, parse_korting_page)
elif args.site == 'housedorf':
    df_src = create_src(args.urls_source, parse_hausedorf_page)
elif args.site == 'dedietrich':
    df_src = create_src(args.urls_source, parse_dedietrich_page)
elif args.site == 'falmec':
    df_src = create_src(args.urls_source, parse_falmec_page)
elif args.site == 'vzug':
    df_src = create_src(args.urls_source, parse_vzug_page)
else:
    raise ValueError("There're no parse function for this site")

df_src, unsyn_set = rename_columns_with_syn_dict(df_src, synonyms_path, all_characteristics_path) # Для обеспечения правильного дописывания данных и для корректного входа к функции, 
                                                                        # генерирующей отчёт
resultdf, com_cols = write_dest(args.input_path, args.output_path, df_src, args.start_row)
resultdf.to_parquet(f"{args.site}_auxiliary.parquet")

com_cols = resultdf.columns.intersection(df_src.columns)
missingdf = df_src.drop(columns=com_cols).copy()
print(unsyn_set)
with open(f'unaccepted_syn_{args.site}.txt', 'w') as f:
    f.write('; '.join(map(str, unsyn_set)))

if args.append:
    append_dataframe_to_excel(missingdf, args.output_path, args.output_path, args.start_row)
else:
    missingdf.insert(0, 'Номенклатура', resultdf['Номенклатура'].copy())
    save_missing(missingdf, f'missing_{args.site}.xlsx')


args = Namespace(start_row=34, append=False, site='falmec', urls_source='urls_falmec.txt', input_path='result_housedorf.xlsx', output_path='result.xlsx')
if not args.output_path:
    raise ValueError("there's not enough arguments")

# Проверка, не повреждены ли входные файлы
wb = load_workbook(args.input_path)

if args.site == 'korting':
    df_src = create_src(args.urls_source, parse_korting_page)
elif args.site == 'housedorf':
    df_src = create_src(args.urls_source, parse_hausedorf_page)
elif args.site == 'dedietrich':
    df_src = create_src(args.urls_source, parse_dedietrich_page)
elif args.site == 'falmec':
    df_src = create_src(args.urls_source, parse_falmec_page)
elif args.site == 'vzug':
    df_src = create_src(args.urls_source, parse_vzug_page)
else:
    raise ValueError("There're no parse function for this site")

df_src, unsyn_set = rename_columns_with_syn_dict(df_src, synonyms_path, all_characteristics_path) # Для обеспечения правильного дописывания данных и для корректного входа к функции, 
                                                                        # генерирующей отчёт
resultdf, com_cols = write_dest(args.input_path, args.output_path, df_src, args.start_row)
resultdf.to_parquet(f"{args.site}_auxiliary.parquet")

com_cols = resultdf.columns.intersection(df_src.columns)
missingdf = df_src.drop(columns=com_cols).copy()
print(unsyn_set)
with open(f'unaccepted_syn_{args.site}.txt', 'w', encoding='utf-8') as f:
    f.write('; '.join(map(str, unsyn_set)))

if args.append:
    append_dataframe_to_excel(missingdf, args.output_path, args.output_path, args.start_row)
else:
    missingdf.insert(0, 'Номенклатура', resultdf['Номенклатура'].copy())
    save_missing(missingdf, f'missing_{args.site}.xlsx')


# Программа generate_syn_report.py

In [19]:
from ruwordnet import RuWordNet
import pymorphy2
import pandas as pd
from tqdm import tqdm
import re
from itertools import product

synonyms_path='synonyms.txt'
all_characteristics_path='all_characteristics.xlsx'

In [10]:
# ---------------------------------------------------------Добавить функции для сопоставления синонимов---------------------------------------------------------

wordnet = RuWordNet()
morph = pymorphy2.MorphAnalyzer()

def get_normal_form(word):
    return morph.parse(word)[0].normal_form

def are_synonyms(word1, word2):
    lemma1 = get_normal_form(word1)
    lemma2 = get_normal_form(word2)

    synsets1 = wordnet.get_synsets(lemma1)
    synsets2 = wordnet.get_synsets(lemma2)

    # Сравниваем наличие общих лемм в синсетах
    for s1 in synsets1:
        for s2 in synsets2:
            if s1.id == s2.id:
                return True
    return False

def list_synonyms_comparison(list1, list2):
    return [are_synonyms(word1, word2) for word1, word2 in zip(list1, list2)]

In [ ]:
# -----------------------------------Добавить функцию для создания отчёта о сопоставлении колонок тем, что уже существуют в 1С-----------------------------------

def parse_custom_dict_line(line):
    """
    Разбирает строку из словаря: <характеристика>: <синоним1>; <синоним2>; ...| <антисиноним1>, <антисиноним2>, ...
    """
    base, *rest = line.strip().split(':')
    if not rest:
        return base.strip(), set(), set()
    syn_ant = rest[0].split('|')
    synonyms = set(map(str.strip, syn_ant[0].split(';'))) if syn_ant[0] else set()
    antonyms = set(map(str.strip, syn_ant[1].split(','))) if len(syn_ant) > 1 else set()
    return base.strip(), synonyms, antonyms

def load_existing_synonyms(file_path):
    syn_dict = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            base, synonyms, antonyms = parse_custom_dict_line(line)
            syn_dict[base] = {'synonyms': synonyms, 'antonyms': antonyms}
    return syn_dict

def tokenize(text):
    return set(re.findall(r'\w+', text.lower()))

def are_words_possibly_synonyms(words1, words2, are_synonims_func):
    pairs = product(words1, words2)
    for w1, w2 in pairs:
        if w1 == w2:
            return True
        if are_synonims_func(w1, w2):
            return True
    return False


def generate_synonym_report(existing_dict_path, all_1c_chars_path, new_chars, are_synonims_func, output_excel_path):
    custom_dict = load_existing_synonyms(existing_dict_path)
    all_chars = pd.read_excel(all_1c_chars_path, header=None).iloc[0].astype(str).tolist()
    result_rows = []

    for c1 in tqdm(all_chars):
        for c2 in new_chars:
            if c1 == c2:
                continue
            tokens1 = tokenize(c1)
            tokens2 = tokenize(c2)
            if are_words_possibly_synonyms(tokens1, tokens2, are_synonims_func):
                result_rows.append((c1, c2, None))  # None для ручной отметки

    df_result = pd.DataFrame(result_rows, columns=["base_char", "compared_char", "label"])
    df_result.to_excel(output_excel_path, index=False)

In [ ]:
# -----------------------------------------------------------------------Запуск генерации-----------------------------------------------------------------------

from argparse import Namespace

args = Namespace(site='korting', synonyms_report_path='syn_report_korting.xlsx')
if not args.synonyms_report_path:
    raise ValueError("there's not enough arguments")

with open(f'unaccepted_syn_{args.site}.txt', 'r', encoding='utf-8') as f:
    data = f.read()

unsyn_list = list(map(str, data.strip().split('; ')))

generate_synonym_report(
    synonyms_path,
    all_characteristics_path,
    unsyn_list,
    are_synonyms,
    args.synonyms_report_path
)

['Срок гарантии (мес.)', 'Габариты (ВхШхГ) (мм)', 'Тип охлаждения холодильной камеры', 'Тип разморозки морозильной камеры', 'Сигнализация открытой двери', 'Приветственный сигнал при открытии дверцы', 'Крепление фасада']


# Программа synonyms_dict_update.py

In [1]:
import pandas as pd
from collections import defaultdict
import os
import argparse
synonyms_path='synonyms.txt'

In [8]:
# --------------------------------------------------------------Обновить словарь синонимов из Excel--------------------------------------------------------------
def load_synonym_dict(dict_path):
    syn_dict = defaultdict(lambda: {"synonyms": set(), "antisynonyms": set()})
    if not os.path.exists(dict_path):
        return syn_dict
    
    with open(dict_path, "r", encoding="utf-8") as f:
        for line in f:
            if ":" not in line:
                continue
            key, rest = line.strip().split(":", 1)
            syn_part, *anti_part = rest.strip().split("|")
            syns = set(map(str.strip, syn_part.strip().split(";"))) if syn_part.strip() else set()
            antis = set(map(str.strip, anti_part[0].strip().split(","))) if anti_part and anti_part[0].strip() else set()
            syn_dict[key.strip()]["synonyms"].update(syns)
            syn_dict[key.strip()]["antisynonyms"].update(antis)
    return syn_dict


def save_synonym_dict(syn_dict, dict_path):
    with open(dict_path, "w", encoding="utf-8") as f:
        for key in sorted(syn_dict.keys()):
            syns = "; ".join(sorted(syn_dict[key]["synonyms"]))
            antis = ", ".join(sorted(syn_dict[key]["antisynonyms"]))
            line = f"{key}: {syns} | {antis}\n"
            f.write(line)


def update_synonym_dict_from_excel(excel_path, dict_path):
    df = pd.read_excel(excel_path)
    if not {"base_char", "compared_char", "label"}.issubset(df.columns):
        raise ValueError("Excel должен содержать столбцы: base_char, compared_char, label")
    
    syn_dict = load_synonym_dict(dict_path)

    for _, row in df.iterrows():
        base = row["base_char"].strip()
        comp = row["compared_char"].strip()
        label = row["label"]

        if label == 1:
            syn_dict[base]["synonyms"].add(comp)
        elif label == 0 or pd.isna(label):
            syn_dict[base]["antisynonyms"].add(comp)
        else:
            continue  # Пропустить некорректные значения

    # Удалить пересекающиеся значения
    for base in syn_dict:
        overlap = syn_dict[base]["synonyms"] & syn_dict[base]["antisynonyms"]
        syn_dict[base]["antisynonyms"] -= overlap

    save_synonym_dict(syn_dict, dict_path)
    print(f"Обновлённый словарь сохранён в {dict_path}")


In [9]:
# Протестировать обновление файла-словаря

from argparse import Namespace

# housedorf
args = Namespace(report_path='syn_report_housedorf.xlsx')
if not args.report_path:
    raise ValueError("there's not enough arguments")

update_synonym_dict_from_excel(args.report_path, synonyms_path)

Обновлённый словарь сохранён в synonyms.txt


# Программа compare.py

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import PatternFill, Alignment
from openpyxl.utils.dataframe import dataframe_to_rows
import argparse
from itertools import product
from ruwordnet import RuWordNet
import pymorphy2
import re

In [18]:
# ---------------------------------------------------------Добавить функции для сопоставления синонимов---------------------------------------------------------

wordnet = RuWordNet()
morph = pymorphy2.MorphAnalyzer()

def get_normal_form(word):
    return morph.parse(word)[0].normal_form

def are_synonyms(word1, word2):
    lemma1 = get_normal_form(word1)
    lemma2 = get_normal_form(word2)

    synsets1 = wordnet.get_synsets(lemma1)
    synsets2 = wordnet.get_synsets(lemma2)

    # Сравниваем наличие общих лемм в синсетах
    for s1 in synsets1:
        for s2 in synsets2:
            if s1.id == s2.id:
                return True
    return False

def tokenize(text):
    return set(re.findall(r'\w+', text.lower()))

def are_words_possibly_synonyms(words1, words2, are_synonims_func):
    pairs = product(words1, words2)
    for w1, w2 in pairs:
        if w1 == w2:
            return True
        if are_synonims_func(w1, w2):
            return True
    return False

In [22]:
# -------------------------------------------------------------------Сравнить записанные данные-------------------------------------------------------------------

def compare_dataframes(df1: pd.DataFrame, df2: pd.DataFrame, name1: str = 'df1', name2: str = 'df2') -> pd.DataFrame:
    """
    Сравнивает два DataFrame по общим столбцам, включая "Номенклатура" как ключ.
    Добавляет префиксы к столбцам (кроме "Номенклатура") и формирует колонку diff_columns.
    """
    if 'Номенклатура' not in df1.columns or 'Номенклатура' not in df2.columns:
        raise ValueError("Оба DataFrame должны содержать колонку 'Номенклатура'")

    # Определим общие характеристики (кроме "Номенклатура")
    common_columns = df1.columns.intersection(df2.columns).difference(['Номенклатура'])

    # Сузим входные DataFrame'ы до нужных колонок
    df1_reduced = df1[['Номенклатура'] + list(common_columns)].copy()
    df2_reduced = df2[['Номенклатура'] + list(common_columns)].copy()

    # Переименуем характеристики с префиксами, "Номенклатура" оставим без изменений
    df1_renamed = df1_reduced.rename(columns={col: f'{name1}_{col}' for col in common_columns})
    df2_renamed = df2_reduced.rename(columns={col: f'{name2}_{col}' for col in common_columns})

    # Объединение по "Номенклатура"
    df_merged = pd.merge(df1_renamed, df2_renamed, on='Номенклатура', how='outer')

    # Функция для сравнения значений по строке
    def get_differences(row):
        diffs = []
        for col in common_columns:
            val1 = row.get(f'{name1}_{col}', None)
            val2 = row.get(f'{name2}_{col}', None)
            if pd.isna(val1) or pd.isna(val2):
                continue
            else:
                tokens1 = tokenize(str(val1))
                tokens2 = tokenize(str(val2))
                if not are_words_possibly_synonyms(tokens1, tokens2, are_synonyms):
                    diffs.append(col)
        return ', '.join(diffs)

    # Добавление столбца с различиями
    df_merged['diff_columns'] = df_merged.apply(get_differences, axis=1)

    return df_merged

# --------------------------------------------------------------Сохранить результат сравнения в excel--------------------------------------------------------------

def save_comparison_to_excel(df: pd.DataFrame, filename: str):
    red_fill = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")
    yellow_fill = PatternFill(start_color="FFEB9C", end_color="FFEB9C", fill_type="solid")

    wb = Workbook()
    ws = wb.active

    # Записываем DataFrame в Excel
    for r in dataframe_to_rows(df, index=False, header=True):
        ws.append(r)

    headers = [cell.value for cell in ws[1]]
    diff_col_index = len(headers)
    prefix_columns = [col for col in headers if col != 'Номенклатура' and col != 'diff_columns']

    # Автонастройка ширины столбцов по первой строке
    for col_idx, cell in enumerate(ws[1], start=1):
        max_length = len(str(cell.value)) if cell.value else 0
        col_letter = cell.column_letter
        ws.column_dimensions[col_letter].width = max_length + 2  # +2 для отступа

    # Применение стилей и переносов
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row, max_col=ws.max_column):
        diff_text = row[diff_col_index - 1].value
        for cell in row:
            cell.alignment = Alignment(wrap_text=True, vertical='center')  # включаем перенос текста

        if isinstance(diff_text, str) and diff_text.strip():
            different_fields = [field.strip() for field in diff_text.split(',')]
            for diff_field in different_fields:
                for col_idx, col_name in enumerate(headers):
                    if col_name.endswith(f"_{diff_field}"):
                        row[col_idx].fill = yellow_fill
        row[diff_col_index - 1].fill = red_fill

    wb.save(filename)

In [23]:
# Протестировать сравнение двух DataFrame и сохранение результата в excel
from argparse import Namespace

args = Namespace(site1='korting', site2='housedorf')
if not args.site2:
    raise ValueError("there's not enough arguments")
if args.site1 == args.site2:
    raise ValueError("the arguments coincide, comparison is impossible")

resultdf_1 = pd.read_parquet(f"{args.site1}_auxiliary.parquet")
resultdf_2 = pd.read_parquet(f"{args.site2}_auxiliary.parquet")
comp_result = compare_dataframes(resultdf_1, resultdf_2, args.site1, args.site2)

save_comparison_to_excel(comp_result, f'comparison_{args.site1}_vs_{args.site2}.xlsx')